In [1]:
import json
import pandas as pd
import numpy as np
from datetime import datetime
from difflib import SequenceMatcher

In [2]:
with open("scraped_results_v2.json", "r") as f:
    results_json = json.load(f)

In [21]:
df = pd.json_normalize(results_json)
df.head(3)

,page_number,winner,loser,cup,stage,boatclass,winner_station,time,date,barrier_time,barrier_loser_leading,fawley_time,fawley_loser_leading,finish_time,verdict
0,1,Oxford Brookes University 'A',Oxford Brookes University 'B',The Prince Albert Challenge Cup,final,Class M4+,Station 1 - Berks,4:30 pm,Sun 7 Jul 24,02:03,False,03:29,False,07:13,4 ½ length
1,1,Oxford Brookes University,Leander Club,The Stewards' Challenge Cup,final,Class M4,Station 1 - Berks,4:20 pm,Sun 7 Jul 24,01:59,True,03:19,False,06:50,2 lengths
2,1,Wycliffe College 'A',Marlow Rowing Club,The Diamond Jubilee Challenge Cup,final,Class W4,Station 1 - Berks,4:10 pm,Sun 7 Jul 24,02:11,False,03:43,False,07:51,4 lengths


In [4]:
# checking that we scraped every page
all(1 <= num <= 378 for num in df['page_number'].unique())

True

In [5]:
# checking that we more or less got the right numbers from each page
# looks like we got a full list from all but the last page which makes sense
# meta value counts!
df['page_number'].value_counts().value_counts()

count
20    377
16      1
Name: count, dtype: int64

In [6]:
# did we someone how pick up an identical line at any point?
df.duplicated().value_counts()
df.loc[df.duplicated() == True]

,page_number,winner,loser,cup,stage,boatclass,winner_station,time,date,barrier_time,barrier_loser_leading,fawley_time,fawley_loser_leading,finish_time,verdict
6233,312,"Commercial Rowing Club, Ireland",The Tideway Scullers' School 'A',The Men's Quadruple Sculls,HEATS,Class M4x,Station 2 - Bucks,2:00 am,Fri 4 Jul 03,1:58,False,3:19,False,7:03,2¾ Lengths


Looks like we've picked up an extra one. Let's delete that. Checking the website, it looks like it's duplicated in the official results.

In [7]:
df = df.drop(df[df.duplicated() == True].index)

## Formatting

Cleaning up the stages column.

In [8]:
df['stage'] = df['stage'].str.lower()
df.loc[df['stage'] == 'semi-finals', 'stage'] = 'semi-final'

The boatclass column.

In [9]:
df['boatclass'] = df['boatclass'].str[6:]

Converting the barrier, fawley, and finish times into seconds.

In [10]:
def convert_to_seconds(time_str):
    if pd.isna(time_str):
        return None
    minutes, seconds = map(int, time_str.split(':'))
    return minutes * 60 + seconds

def clean_times(column_name):
    df[column_name] = df[column_name].replace({'NTT' : None,'' : np.nan})
    df[column_name] = df[column_name].str.replace("=", "")
    df[column_name] = df[column_name].str.replace(",", ":")
    df[column_name] = df[column_name].apply(convert_to_seconds)
    
clean_times('barrier_time')
clean_times('fawley_time')
clean_times('finish_time')

Time of day to 24h. Note that most of these results are just labelled as 2am, which obviously isn't correct. I've replaced these with None values.

In [11]:
def convert_time_to_24hr(time_str):
    hour, minute = map(str, time_str.split(":"))
    minute, time_of_day = map(str, minute.split(" "))
    
    if time_of_day == "pm" and hour != "0":
        hour = int(hour) + 12
    elif time_of_day == "pm" and hour == "0":
        hour = 12    
    elif time_of_day == "am" and len(hour) == 1:
        hour = "0" + hour
    
    converted_time = str(hour) + ":" + minute
    
    # a lot of races have been put at 02:00 - presumably the time wasn't recorded
    if converted_time == "02:00":
        converted_time = None
    
    return converted_time

df['time'] = df['time'].apply(convert_time_to_24hr)

Converting the lengths to something usable

In [12]:
def convert_verdict_to_lengths(verdict):
    
    length_mapping = {'¼' : '.25', '⅓' : '.33', '½' : '.5', '⅔' : '.66', '¾' : '.75'}
    
    verdict = verdict.lower()
    
    verdict = verdict.replace('lengths', 'length')
    verdict = verdict.replace('lenths', 'length')
    verdict = verdict.replace('lenghts', 'length')
    
    for k, v in length_mapping.items():
        if k in verdict:
            verdict = verdict.replace(k, v)
    
    verdict = verdict.replace(' ', '')
    
    try:
        int(verdict[-1])
        verdict = verdict + 'l'
    except:
        pass
    
    
    if 'length' in verdict:
        verdict = verdict.replace('length', 'l')
        
                    
    if verdict.startswith('.'):
        verdict = '0' + verdict
        
    
    if 'feet' in verdict:
        verdict = verdict.replace('feet', 'ft')
        
    elif 'foot' in verdict:
        verdict = verdict.replace('foot', 'ft')    
    
    elif 'disqualified' in verdict:
        verdict = 'disqualified'
        
    elif 'nro' in verdict:
        verdict = 'not rowed out'
        
    elif verdict == 'sculledover':
        verdict = 'rowed over'
    elif verdict == 'rowedover':
        verdict = 'rowed over'
    elif verdict == 'notrowedout':
        verdict = 'not rowed out'
    elif verdict == 'canvas':
        verdict = '0.33l'
    elif verdict == 'easiy':
        verdict = 'easily'
        
    if verdict.endswith('l'):
        verdict = verdict.replace('l', ' lengths')
        
        

    return verdict

df['verdict'] = df['verdict'].apply(convert_verdict_to_lengths)

Change the date to datetime format

In [13]:
df['date'].unique()

def to_datetime(date_string):
    date_split = date_string.split()
    date_string = (' ').join(date_split[1:])
    date_obj = datetime.strptime(date_string, '%d %b %y')
    
    return date_obj
    
df['date'] = df['date'].apply(to_datetime)


Let's also extract the club names.

In [22]:
df['winner'].unique()

def get_club(boat_name):
    internal_rankings = [" 'A'", " 'B'", " 'C'", " 'D'", " 'E'", " 'F'", " 'G'", " 'H'", " 'I'"]
    
    club_name = boat_name
    
    
    for rank in internal_rankings:
        if rank in boat_name:
            club_name = boat_name.replace(rank, '')
            
    club_name = club_name.strip()


    return club_name

df['winner'] = df['winner'].str.strip()
df['winning_club'] = df['winner'].apply(get_club)

df['loser'] = df['loser'].str.strip()
df['losing_club'] = df['loser'].apply(get_club)

df = df.iloc[:, [0, 1, 15, 2, 16, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]]

The final table.

In [24]:
df.head(3)

,page_number,winner,winning_club,loser,losing_club,cup,stage,boatclass,winner_station,time,date,barrier_time,barrier_loser_leading,fawley_time,fawley_loser_leading,finish_time,verdict
0,1,Oxford Brookes University 'A',Oxford Brookes University,Oxford Brookes University 'B',Oxford Brookes University,The Prince Albert Challenge Cup,final,Class M4+,Station 1 - Berks,4:30 pm,Sun 7 Jul 24,02:03,False,03:29,False,07:13,4 ½ length
1,1,Oxford Brookes University,Oxford Brookes University,Leander Club,Leander Club,The Stewards' Challenge Cup,final,Class M4,Station 1 - Berks,4:20 pm,Sun 7 Jul 24,01:59,True,03:19,False,06:50,2 lengths
2,1,Wycliffe College 'A',Wycliffe College,Marlow Rowing Club,Marlow Rowing Club,The Diamond Jubilee Challenge Cup,final,Class W4,Station 1 - Berks,4:10 pm,Sun 7 Jul 24,02:11,False,03:43,False,07:51,4 lengths


Exporting to csv.

In [17]:
df.to_csv('henley_results_cleaned.csv', index=False)  